# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

# Load environment variables
%load_ext dotenv
%dotenv

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Get the path from the environment variable `PRICE_DATA`
PRICE_DATA = os.getenv("PRICE_DATA")

# Get the path of all parquet files in the directory `PRICE_DATA`
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

# Read the parquet files
dd_pq = dd.read_parquet(parquet_files)

# Add the lag for Close
dd_lag_close = dd_pq.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

# Add the lag for Adj_Close
dd_lag_adj_close = dd_lag_close.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

# Add the returns
dd_returns = dd_lag_adj_close.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Returns = lambda x: x['Close']/x['Close_lag_1'] - 1)
)

# Add the range
dd_hi_lo_range = dd_returns.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Hi_Lo_Range = lambda x: x['High'] - x['Low'])
)

# Check the result
dd_hi_lo_range.compute().head()

/var/folders/d3/tbj59bxx5m79k_ztyd_k3_3h0000gn/T/ipykernel_3809/3022789718.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lag_close = dd_pq.groupby('Ticker', group_keys=False).apply(
/var/folders/d3/tbj59bxx5m79k_ztyd_k3_3h0000gn/T/ipykernel_3809/3022789718.py:12: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lag_adj_close = dd_lag_close.groupby('Ticker', group_keys=False).apply(
/var/folders/d3/tbj59bxx5m79k_ztyd_k3_3h0000gn/T/ipykernel_3809/3022789718.py:17: UserWarning: `meta` is not specified, inferred

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range
Ticker,,,,,,,,,,,,
DOV,2002-01-02,NaN,16.045649,16.123581,15.772880,16.062966,1022622.0,2002,NaN,NaN,NaN,0.350701
DOV,2002-01-03,NaN,16.340065,16.409339,16.041319,16.067297,989355.0,2002,16.045649,NaN,0.018349,0.368020
DOV,2002-01-04,NaN,16.721079,16.764375,16.409345,16.409345,1319488.0,2002,16.340065,NaN,0.023318,0.355031
DOV,2002-01-07,NaN,16.608505,16.881273,16.591187,16.777361,1060812.0,2002,16.721079,NaN,-0.006732,0.290086
DOV,2002-01-08,NaN,16.236166,16.669131,16.062980,16.582537,976377.0,2002,16.608505,NaN,-0.022419,0.606151


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.

# Convert the Dask data frame to a pandas data frame. 
pd_df = dd_hi_lo_range.compute()

# Add the moving average of `Returns` feature using a window of 10 days.
dd_feat = pd_df.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(MA_Returns = x['Returns'].rolling(10).mean())
)

# Check the result
pd_df.head(30)

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range
Ticker,,,,,,,,,,,,
DOV,2005-01-03,NaN,18.734228,19.157318,18.693283,19.075430,762305.0,2005,NaN,NaN,NaN,0.464035
DOV,2005-01-04,NaN,18.647797,19.011745,18.597753,18.911659,1330378.0,2005,18.734228,NaN,-0.004614,0.413992
DOV,2005-01-05,NaN,18.283848,18.857067,18.261102,18.647797,1176575.0,2005,18.647797,NaN,-0.019517,0.595965
DOV,2005-01-06,NaN,18.110964,18.311135,18.070020,18.311135,1395122.0,2005,18.283848,NaN,-0.009456,0.241114
DOV,2005-01-07,NaN,17.847097,18.138255,17.833449,18.070016,1192984.0,2005,18.110964,NaN,-0.014569,0.304807
DOV,2005-01-10,NaN,17.742468,17.915344,17.742468,17.769764,1475977.0,2005,17.847097,NaN,-0.005863,0.172876
DOV,2005-01-11,NaN,17.605989,17.783414,17.555946,17.747019,1654097.0,2005,17.742468,NaN,-0.007692,0.227468
DOV,2005-01-12,NaN,17.988131,17.988131,17.642380,17.674225,1825802.0,2005,17.605989,NaN,0.021705,0.345750
DOV,2005-01-13,NaN,17.833454,18.038175,17.774313,17.988133,898803.0,2005,17.988131,NaN,-0.008599,0.263862


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
  
  > I would say no, it was not necessary to convert to pandas to calculate the moving average return.
+ Would it have been better to do it in Dask? Why?
  
  > Converting to pandas could be useful to be able to access the full dataset, but it's a slow solution, because
  >  - Pandas does that sequentially, while Dask can process multiple chunks of data at the same time , which in turn means that that data can be processed in parallel on multiple cores, and, at scale, on multiple machines.
  >  - Pandas does not scale well with the size of the dataset.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [x] Created a branch with the correct naming convention.
- [x] Ensured that the repository is public.
- [x] Reviewed the PR description guidelines and adhered to them.
- [x] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-5-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.